In [1]:
# Import Splinter and BeautifulSoup

from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# set up the executable path

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# assign the url and instruct the browser to visit

# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)

# we're searching for elements with a specific combination of tag (div) and attribute (list_text)
# Optional delay for loading the page - wait time of 1 second
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# set up the html parser

html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [5]:
# Notice how we've assigned slide_elem as the variable to look for the <div /> tag and its descendent 
# (the other tags within the <div /> element)? This is our parent element. This means that this element holds all 
# of the other elements within it, and we'll reference it when we want to filter search results even further. 
# The . is used for selecting classes, such as list_text, so the code 'div.list_text' pinpoints the <div /> tag 
# with the class of list_text. CSS works from right to left, such as returning the last item on the list instead 
# of the first. Because of this, when using select_one, the first matching element returned will be a <li /> element 
# with a class of slide and all nested elements within it.

In [6]:
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA Readies Perseverance Mars Rover's Earthly Twin </div>

In [7]:
# In this line of code, we chained .find onto our previously assigned variable, slide_elem. When we do this, we're 
# saying, "This variable holds a ton of information, so look inside of that information to find this specific data." 
# The data we're looking for is the content title, which we've specified by saying, "The specific data is in a 
# <div /> with a class of 'content_title'."

# The output should be the HTML containing the content title and anything else nested inside of that <div />.

In [8]:
# need to just get the title though, minus the html language

# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA Readies Perseverance Mars Rover's Earthly Twin "

In [9]:
# We've added something new to our .find() method here: .get_text(). When this new method is chained onto .find(), 
# only the text of the element is returned. The code above, for example, would return only the title of the news 
# article and not any of the HTML tags or elements.

In [10]:
# IMPORTANT
# There are two methods used to find tags and attributes with BeautifulSoup:
# .find() is used when we want only the first class and attribute we've specified.
# .find_all() is used when we want to retrieve all of the tags and attributes.
#For example, if we were to use .find_all() instead of .find() when pulling the summary, we would retrieve all of 
#the summaries on the page instead of just the first one.

In [11]:
# get the text from the shot article summary
# Use the parent element to find the paragraph text

news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape."

In [12]:
# 10.3.4 scraping full size image

### Featured Images

In [13]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [14]:
# Find and click the full image button
# [1] indicates that we want the second button (FULL IMAGE)
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [15]:
# Parse the resulting html with soup
# With the new page loaded onto our automated browser, it needs to be parsed so we can continue and scrape the 
# full-size image URL. 
html = browser.html
img_soup = soup(html, 'html.parser')

In [16]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [17]:
# SRC - It's important to note that the value of the src will be different every time the page is updated, so we can't 
# simply record the current value—we would only pull that image each time the code is executed, instead of the 
#most recent one.

In [18]:
# 1) An img tag is nested within this HTML, so we've included it.
# 2) .get('src') pulls the link to the image.
# What we've done here is tell BeautifulSoup to look inside the <img /> tag for an image with a class of 
#fancybox-image.

In [19]:
# the image link is only partial, need to make it complete
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [20]:
# scraping an entire table with read.html()
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [21]:
# With the first line, we're creating a new DataFrame from the HTML table. The Pandas function read_html() 
# specifically searches for and returns a list of tables found in the HTML. By specifying an index of 0, we're 
# telling Pandas to pull only the first table it encounters, or the first item in the list. Then, it turns the table 
# into a DataFrame.
# in the second line, we assign columns to the new DataFrame for additional clarity.
# finally, by using the .set_index() function, we're turning the Description column into the DataFrame's index. 
# inplace=True means that the updated index will remain in place, without having to reassign the DataFrame to a 
#new variable.

In [22]:
# need to convert the dataframe back to html in order to use it in our webpage
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [23]:
# need to turn off the scraping code when we're done
browser.quit()